# EJECUTOR
En este notebook construiremos la logica del script que ejecutará las ordenes de mercado que el modelo tome como operaciones de alta probabilidad
Aquí no se profundizará en los detalles, porque esta es la aplicación del proyecto que hemos venido construyendo.
El objetivo es usar el modelo construido para predecir ordenes en el mercado.

IMPORTAMOS LIBRERÍAS NECESARIAS

In [1]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model


INICIALIZAMOS LA PLATAFORMA

In [3]:

# start the platform with initialize()
mt5.initialize()

True

HACEMOS LOGIN EN LA CUENTA DE TRADING

In [4]:
#Esta es una cuenta demo de MT5
login = 1520445584
password = 'SB?p?A7tE3'
server = 'FTMO-Demo2'

columns_order=['open', 'high', 'low', 'close', 'tick_volume', 'Hour','maximo', 'minimo', 'ATR14', 'RSI14']

mt5.login(login, password, server)

True

CARGAMOS LOS MODELOS YA ENTRENADOS

In [6]:
Buys_model = load_model('Aplicación/models/Buys_model.keras')
Sells_model = load_model('Aplicación/models/Sells_model.keras')

CARGAMOS LAS LAS FUNCIONES QUE USAREMOS PARA ACTUALIZAR LOS DATOS DESCARGADOS EN TIEMPO REAL DE LOS SERVIDORES DE LA CUENTA DE TRADING DEMO

In [9]:
def get_last_candles(symbol, timeframe, num_candles):
    """Obtiene las últimas 'num_candles' velas para el símbolo y timeframe especificados."""
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_candles+13)#13 velas extras para el ATR y RSI
    if rates is None:
        print(f"No se pudieron obtener las velas para {symbol}.")
        return None
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.index=df['time']
    df.index = pd.to_datetime(df.index)  # Convertir a datetime
    df=df.drop('time',axis=1)
    df.index=df.index.tz_localize('Etc/GMT-3')
    df.index=df.index.tz_convert('Etc/GMT+5').tz_localize(None)
    df['DayOfYear'] = df.index.dayofyear
    df['Hour'] = df.index.hour
    df=agregar_max_min(df,4)#agregar maximos y minimos
    df=agregar_atr_rsi(df, 14, 14)#agregar ATR y RSI
    df = df[columns_order]
    return df[13:-1]
def agregar_atr_rsi(df, periodo_atr, periodo_rsi):
    """
    Agrega columnas ATR y RSI al DataFrame, calculadas por día del año.

    Parámetros:
    - df: pandas DataFrame con columnas ['open', 'high', 'low', 'close', 'tick_volume', 
                                         'DayOfYear', 'Hour', 'Minute', 'Tres']
    - periodo_atr: int, el período para calcular el ATR
    - periodo_rsi: int, el período para calcular el RSI

    Retorna:
    - pandas DataFrame con nuevas columnas 'ATR{periodo_atr}' y 'RSI{periodo_rsi}'
    """
    # Asegurar que el DataFrame está ordenado correctamente
    #df = df.sort_values(['DayOfYear', 'Hour', 'Minute', 'Tres']).reset_index(drop=True)

    ### Cálculo del ATR ###

    # Calcular el Close anterior por día
    df['prev_close'] = df.groupby('DayOfYear')['close'].shift(1)

    # Calcular el True Range (TR)
    df['TR'] = df.apply(
        lambda row: max(
            row['high'] - row['low'],
            abs(row['high'] - row['prev_close']) if not pd.isna(row['prev_close']) else row['high'] - row['low'],
            abs(row['low'] - row['prev_close']) if not pd.isna(row['prev_close']) else row['high'] - row['low']
        ),
        axis=1
    )

    # Definir el nombre de la nueva columna ATR
    atr_column = f'ATR{periodo_atr}'

    # Calcular el ATR como media móvil simple del TR por grupo (día)
    df[atr_column] = df.groupby('DayOfYear')['TR'].transform(
        lambda x: x.rolling(window=periodo_atr, min_periods=periodo_atr).mean()
    )

    # Reemplazar los valores NaN (primeras observaciones) por 0
    df[atr_column] = df[atr_column].fillna(0)

    # Redondear el ATR para mayor claridad
    df[atr_column] = df[atr_column].round(4)

    ### Cálculo del RSI ###

    # Definir el nombre de la nueva columna RSI
    rsi_column = f'RSI{periodo_rsi}'

    # Definir la función calculate_rsi dentro del ámbito para acceder a 'periodo_rsi'
    def calculate_rsi(close_series):
        delta = close_series.diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)

        # Usar EMA para promedio de ganancias y pérdidas
        avg_gain = gain.ewm(alpha=1/periodo_rsi, min_periods=periodo_rsi, adjust=False).mean()
        avg_loss = loss.ewm(alpha=1/periodo_rsi, min_periods=periodo_rsi, adjust=False).mean()

        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))

        # Reemplazar NaN iniciales por 0
        rsi = rsi.fillna(0)

        return rsi

    # Calcular el RSI por grupo (día), seleccionando sólo la columna 'close'
    df[rsi_column] = df.groupby('DayOfYear', group_keys=False)['close'].apply(calculate_rsi)

    # Redondear el RSI para mayor claridad
    df[rsi_column] = df[rsi_column].round(2)

    # Eliminar columnas auxiliares
    df = df.drop(columns=['prev_close', 'TR'])

    return df
def agregar_max_min(df, n):
    """
    Agrega las columnas 'maximo' y 'minimo' al DataFrame.
    
    Parámetros:
    - df (pd.DataFrame): DataFrame que contiene las columnas ['open', 'high', 'low', 'close'].
    - n (int): Número de observaciones anteriores y posteriores a considerar.
    
    Retorna:
    - pd.DataFrame: DataFrame original con las columnas 'maximo' y 'minimo' agregadas.
    """
    # Asegurarse de que el DataFrame está ordenado correctamente (opcional)
    # df = df.sort_index()

    # Calcular el máximo en una ventana de 2n+1 centrada en cada observación
    rolling_max = df['high'].rolling(window=2*n+1, center=True).max()
    # Asignar 1 si el 'high' actual es igual al máximo de la ventana, de lo contrario 0
    df['maximo'] = (df['high'] == rolling_max).astype(int)
    
    # Calcular el mínimo en una ventana de 2n+1 centrada en cada observación
    rolling_min = df['low'].rolling(window=2*n+1, center=True).min()
    # Asignar 1 si el 'low' actual es igual al mínimo de la ventana, de lo contrario 0
    df['minimo'] = (df['low'] == rolling_min).astype(int)
    
    # Opcional: Reemplazar NaN en las primeras y últimas n filas con 0
    df['maximo'] = df['maximo'].fillna(0).astype(int)
    df['minimo'] = df['minimo'].fillna(0).astype(int)
    
    return df
def predict_operations(data):
    """Realiza las predicciones de compra y venta."""
    y_pred_prob_Buys=Buys_model.predict(data)[0]
    y_pred_prob_Sells=Sells_model.predict(data)[0]
    buy_pred = (y_pred_prob_Buys >= 0.7).astype(int).reshape(-1)
    sell_pred = (y_pred_prob_Sells >= 0.7).astype(int).reshape(-1)
    return y_pred_prob_Buys,y_pred_prob_Sells,buy_pred, sell_pred
def prepare_data(df,scaler_trained=None,columns_=None):
    try:
        df[columns_] = scaler_trained.transform(df[columns_])
    except:
        print('hubo un error al cargar el scaler o cargar las columnas')
    return np.expand_dims(df.values, axis=0)

EXTRAEMOS LAS ULTIMAS 300 VELAS DEL MERCADO

In [12]:
data=get_last_candles('US30.cash', mt5.TIMEFRAME_M1, 300)
data.tail()

,open,high,low,close,tick_volume,Hour,maximo,minimo,ATR14,RSI14
time,,,,,,,,,,
2024-10-09 09:36:00,42150.15,42182.65,42147.65,42181.65,199,9,0,0,17.8571,49.74
2024-10-09 09:37:00,42182.65,42194.15,42180.15,42193.15,173,9,0,0,17.2071,53.77
2024-10-09 09:38:00,42192.65,42194.15,42181.65,42189.65,174,9,0,0,17.2071,52.39
2024-10-09 09:39:00,42190.65,42205.15,42189.65,42198.65,140,9,0,0,16.8143,55.54
2024-10-09 09:40:00,42199.15,42204.65,42189.65,42193.65,172,9,0,0,16.7429,53.42


CARGAMOS EL DATAFRAME QUE USAREMOS PARA ENTRENAR EL ESCALADOR DE CARACTERISTICAS

In [18]:
Buys_sample= pd.read_csv('Buys.csv')

ENTRENAMOS EL ESCALADOR

In [19]:


columnas_a_normalizar = ['open', 'high', 'low', 'close', 'tick_volume', 'Hour', 'ATR14', 'RSI14']
# Crear un objeto de MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(Buys_sample[data.columns][columnas_a_normalizar])  # Ajusta el escalador con los datos de Buys

df_normalized=prepare_data(data,scaler_trained=scaler,columns_=columnas_a_normalizar)


PREDECIMOS UN LA PROBABILIDAD
- Esta es en pocas lineas de codigo la logica que sigue la aplicación para ejecutar las ordenes de mercado
- hasta ahora hemos predicho una secuencia, pero en la aplicación este proceso se repite hasta encontrar un punto de alta probabilidad y ejecutará la orden de mercado

In [31]:
b_r,s_r,buy_pred, sell_pred = predict_operations(df_normalized)
if buy_pred == 0 and sell_pred == 0 :
    print("predicción insuficiente  venta: {} compra: {}".format(s_r,b_r))
if buy_pred == 1 and sell_pred == 0 :
    print("Predicción de Compra detectada. Abriendo orden de compra.")
    #open_order(SYMBOL, mt5.ORDER_TYPE_BUY, STOP_LOSS_PIPS, STOP_LOSS_PIPS * TAKE_PROFIT_MULTIPLIER)
if sell_pred == 1 and buy_pred == 0:
    print("Predicción de Venta detectada. Abriendo orden de venta.")
    #open_order(SYMBOL, mt5.ORDER_TYPE_SELL, STOP_LOSS_PIPS, STOP_LOSS_PIPS * TAKE_PROFIT_MULTIPLIER)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
predicción insuficiente  venta: [0.00144813] compra: [0.00014596]
